In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
import datetime as dt
import streamlit as st

In [33]:
class Blackscholes:
    def __init__(self,Strike,Stock,Time,RFR,Vol=None):
        ''' Basic Black Scholes calculator '''
        args = [Strike,Stock,Time,Vol,RFR]
        for i in args :
            if not isinstance(i,(int,float,dict)):
                raise Exception('Please check the types of the arguments, only float and int are accepted as valid arguments')
        else:   
            self.strike = Strike
            self.stock = Stock
            self.time = Time
            self.vol = Vol
            self.rf = RFR

    def __calculate_d_option(self):
        pass

    def __Calculate_price(self):
        pass


In [35]:
class call_option(Blackscholes):
    '''Call option class'''
    def __init__(self,Strike,Time,RFR,populate=False,Stock=None,ticker=None,Vol=None):
        if populate:
            self.strike = Strike
            self.time = Time
            self.ticker = ticker
            self.rf = RFR
            self.data = yf.download(self.ticker,start=dt.datetime.now() - dt.timedelta(365),end=dt.datetime.now())
            self.stock = np.round(self.data['Close'].iloc[-1,0],4)
            self.vol = np.round(self.data['Close'].pct_change().dropna().std().iloc[0],4)
        else:
            super().__init__(Strike, Stock, Time, RFR,Vol)
    
    def calculate_d_call(self):
        d1 = (np.log(self.stock / self.strike) + (self.rf + (self.vol ** 2) / 2) * self.time) / (self.vol * np.sqrt(self.time))
        d2 = d1 - self.vol * np.sqrt(self.time)
        return d1,d2
    
    def Calculate_price_call(self):
        d1 = self.calculate_d_call()[0]
        d2 = self.calculate_d_call()[1]
        price = self.stock * norm.cdf(d1) - ((self.strike * np.exp(-self.rf*self.time))*norm.cdf(d2))
        return price
    

In [36]:
class put_option(Blackscholes):
    '''Put option class'''
    def __init__(self,Strike,Time,RFR,populate=False,Stock=None,ticker=None,Vol=None):
        if populate:
            self.strike = Strike
            self.time = Time
            self.ticker = ticker
            self.rf = RFR
            self.data = yf.download(self.ticker,start=dt.datetime.now() - dt.timedelta(365),end=dt.datetime.now())
            self.stock = np.round(self.data['Close'].iloc[-1,0],4)
            self.vol = np.round(self.data['Close'].pct_change().dropna().std().iloc[0],4)
        else:
            super().__init__(Strike, Stock, Time, RFR,Vol)
    
    def calculate_d_put(self):
        d1 = (np.log(self.stock / self.strike) + (self.rf + (self.vol ** 2) / 2) * self.time) / (self.vol * np.sqrt(self.time))
        d2 = d1 - self.vol * np.sqrt(self.time)
        return d1,d2
    
    def Calculate_price_put(self):
        d1 = self.calculate_d_put()[0]
        d2 = self.calculate_d_put()[1]
        price = self.strike * np.exp(-self.rf * self.time) * norm.cdf(-d2) - self.stock * norm.cdf(-d1)
        return price